In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Fine-tune a pretrained model

## Prepare a dataset

In [ ]:
!pip install emoji==1.6.0

In [ ]:
!pip install sacremoses

In [ ]:
import re
import emoji
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


In [ ]:
!wget 2019.poleval.pl/task6/task_6-1.zip
!unzip task_6-1.zip

In [ ]:
def makeReadable(line):
  line = line.replace("@anonymized_account", '')
  line = line.replace("RT", '')
  line = line.replace("\n", '')
  line = line.replace("\\", '')
  line = emoji.get_emoji_regexp().sub(u'', line)

  urlRegex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  line = re.sub(urlRegex, '', line)
  line = line.replace(r"https://", '')

  line = re.sub("#(\w+)", '', line)

  line = re.sub('^\s*', '', line)

  return line

In [ ]:
zdania = []
max_length = 0
with open ("training_set_clean_only_text.txt", "r", encoding='utf8') as textFile:
   for line in textFile:
     line = makeReadable(line)   
     #print (line)
     zdania.append(line)
     if len(line.split()) > max_length:
       max_length = len(line.split())

print(len(zdania))
print(max_length)

In [ ]:
klasy = []
with open ("training_set_clean_only_tags.txt", "r", encoding='utf8') as textFile:
   for line in textFile:
    klasy.append(int(line))

print(len(klasy))  

In [ ]:
listOfSentencesPlusTag = []

for x in range (len(zdania)):
  if len(zdania[x]) == 2: #usuniecie pozycji, które zawierają jedynie "?"    
    zdania.remove(zdania[x])
    klasy.remove(klasy[x])

In [ ]:
df = pd.DataFrame(list(zip(zdania, klasy)),
               columns =["text", "label"])

In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

dataset[0]

In [ ]:
type(dataset)

As you now know, you need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths. To process your dataset in one step, use 🤗 Datasets [`map`](https://huggingface.co/docs/datasets/process.html#map) method to apply a preprocessing function over the entire dataset:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
len(tokenized_datasets)

If you like, you can create a smaller subset of the full dataset to fine-tune on to reduce the time it takes:

In [ ]:
final_dataset = tokenized_datasets.train_test_split(test_size=0.1)

In [ ]:
small_train_dataset = final_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = final_dataset["test"].shuffle(seed=42).select(range(1000))


<a id='trainer'></a>

## Train

🤗 Transformers provides a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class optimized for training 🤗 Transformers models, making it easier to start training without manually writing your own training loop. The [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision.

Start by loading your model and specify the number of expected labels. From the Yelp Review [dataset card](https://huggingface.co/datasets/yelp_review_full#data-fields), you know there are five labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("allegro/herbert-base-cased")

<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### Training hyperparameters

Next, create a [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training [hyperparameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments), but feel free to experiment with these to find your optimal settings.


### Wandb

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()


In [ ]:
!WANDB_PROJECT=cyberbullying_detection

### Metrics

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) does not automatically evaluate model performance during training. You will need to pass [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) a function to compute and report metrics. The 🤗 Datasets library provides a simple [`accuracy`](https://huggingface.co/metrics/accuracy) function you can load with the `load_metric` (see this [tutorial](https://huggingface.co/docs/datasets/metrics.html) for more information) function.

Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits (remember all 🤗 Transformers models return logits):

In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch", 
                                  num_train_epochs=3, 
                                  report_to="wandb",  # enable logging to W&B
                                  run_name="herbert-base"  # name of the W&B run (optional)
)

### Trainer

Create a [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) object with your model, training arguments, training and test datasets, and evaluation function:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,

)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trainer.train()

<a id='keras'></a>